In [1]:
#This notebook takes a previously trained RNN and evaluates it on held-out data, saving the outputs for later processing.
#We also compute here the overall character error rate and word error rate across all held-out data. 

In [2]:
import tensorflow as tf
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()

#suppress all tensorflow warnings (largely related to compatability with v2)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import numpy as np
import scipy.io
import os
import matplotlib.pyplot as plt
from datetime import datetime
from charSeqRNN import charSeqRNN, getDefaultRNNArgs
from characterDefinitions import getHandwritingCharacterDefinitions
from characterDefinitionsOrig import getHandwritingCharacterDefinitionsOrig
import tensorflow as tf

from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

#point this towards the top level dataset directory
rootDir = os.path.expanduser('.') + '/handwritingBCIData/'

#evaluate the RNN on these datasets
dataDirs = ['t5.2019.05.08','t5.2019.11.25','t5.2019.12.09','t5.2019.12.11','t5.2019.12.18',
            't5.2019.12.20','t5.2020.01.06','t5.2020.01.08','t5.2020.01.13','t5.2020.01.15',
            'IamOnline1','IamOnline2','IamOnline3', 
            'IamOnline5','IamOnline6']
dataDirs = ['t5.2019.05.08','t5.2019.11.25','t5.2019.12.09','t5.2019.12.11','t5.2019.12.18',
            't5.2019.12.20','t5.2020.01.06','t5.2020.01.08']
dataDirs = ['t5.2019.05.08','t5.2019.11.25','t5.2019.12.09','t5.2019.12.11','t5.2019.12.18',
            't5.2019.12.20','t5.2020.01.06','t5.2020.01.08','t5.2020.01.13','t5.2020.01.15']
dataDirs = ['IamOnline1','IamOnline2', 'IamOnline5']
#use this train/test partition
cvPart = 'HeldOutTrials'

#point this towards the specific RNN we want to evaluate
rnnOutputDir = cvPart

#this prevents tensorflow from taking over more than one gpu on a multi-gpu machine
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='0'

#this is where we're going to save the RNN outputs
inferenceSaveDir = rootDir+'RNNTrainingSteps/Step5_RNNInference/' + rnnOutputDir

if not os.path.isdir(rootDir + 'RNNTrainingSteps/Step5_RNNInference'):
    os.mkdir(rootDir + 'RNNTrainingSteps/Step5_RNNInference')
    
if not os.path.isdir(inferenceSaveDir):
    os.mkdir(inferenceSaveDir)
    

In [3]:
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())
#pip install pyzmq==19.0.2
#Configures the RNN for inference mode.
args = getDefaultRNNArgs()

args['outputDir'] = rootDir+'RNNTrainingSteps/Step4_RNNTraining/'+rnnOutputDir
args['loadDir'] = args['outputDir']
args['mode'] = 'infer'
args['timeSteps'] = 7500 #Need to specify enough time steps so that the longest sentence fits in the minibatch
args['batchSize'] = 2 #Process just two sentences at a time, to make sure we have enough memory
args['synthBatchSize'] = 0 #turn off synthetic data here, we are only using real data

#Proceeds one dataset at a time. Currently the code is setup to only process a single dataset at inference time,
#so we have to rebuild the graph for each dataset.
for x in range(len(dataDirs)):
    #configure the RNN to process this particular dataset
    print(' ')
    print('Processing dataset ' + dataDirs[x])
    
    #defines the list of all 31 characters and what to call them
    if "IamOnline" in dataDirs[x]:
       charDef = getHandwritingCharacterDefinitions()
    else:
       charDef = getHandwritingCharacterDefinitionsOrig() 
    
    if "IamOnline" in dataDirs[x]:
  #    sentenceDat = scipy.io.loadmat(rootDir+'Datasets/'+'t5.2019.05.08'+'/sentences.mat')# Template is required
      sentence1 = scipy.io.loadmat(rootDir+'Datasets/'+dataDirs[x]+'/'+'neuralCubeStruct_IamOnline.mat')
      sentenceDat = scipy.io.loadmat(rootDir+'Datasets/'+dataDirs[x]+'/'+'neuralCubeStruct_IamOnline.mat')
  #    sentenceDat['neuralActivityCube'] = sentence1['neuralActivityCube']
  #    sentenceDat['intendedText'] = sentence1['intendedText']
  #    sentenceDat['sentencePrompt'] = sentence1['sentencePrompt']
  #    sentenceDat['numTimeBinsPerSentence'] = sentence1['numTimeBinsPerSentence']
      sentenceDat['blockList'] = []
#      sentenceDat['trainPartitionIdx'] = sentence1['trainPartitionIdx']
#      sentenceDat['testPartitionIdx'] = sentence1['testPartitionIdx'] 
    else:
      sentenceDat = scipy.io.loadmat(rootDir+'Datasets/'+dataDirs[x]+'/sentences.mat')
    
    scipy.io.savemat('local/'+dataDirs[x]+'_sentences.mat', sentenceDat) 
    
    if "IamOnline" in dataDirs[x]:
 #      singleLetterDat =  scipy.io.loadmat(rootDir+'Datasets/'+'t5.2019.05.08'+'/singleLetters.mat')   # Template is required to overwrite
       singleLetterDat = scipy.io.loadmat(rootDir+'Datasets/'+dataDirs[x]+'/'+'singleChar_IamOnline.mat') # with new data
       print("Modifying to Iamonline data")
#       for char in charDef['charList']:
#         singleLetterDat['neuralActivityCube_'+char] = single1['neuralActivityCube_'+char].astype(np.float64)
    else:
       singleLetterDat = scipy.io.loadmat(rootDir+'Datasets/'+dataDirs[x]+'/singleLetters.mat')
    
        
    trainTest = scipy.io.loadmat(rootDir+'RNNTrainingSteps/trainTestPartitions_'+cvPart+'.mat')
    if "IamOnline" in dataDirs[x]: 
      print("Working on ", dataDirs[x])
      alist = sentence1['trainPartitionIdx']
      print("Before Iamonline: ", alist)
      print("Before Neural", trainTest['t5.2019.05.08_train'])
      indices=np.argwhere(np.isin(alist,trainTest['t5.2019.05.08_test'])) # Delete test indices from the train
      alist=[np.delete(alist,indices)]
      trainTest[dataDirs[x]+'_train'] = alist
      trainTest[dataDirs[x]+'_train'] = sentence1['trainPartitionIdx1']
      trainTest[dataDirs[x]+'_test'] = sentence1['testPartitionIdx1']
    else:
      print("Before", trainTest[dataDirs[x]+'_train']) # Original samples in the dataset
     # nueral_train = np.array([1, 8, 10, 15,20, 30, 40, 50]) # Samples used for training
    #  nueral_train =  np.array([1, 8, 35, 45])   
      maxtrainset = np.array([[0, 5, 8, 10, 12, 15, 17, 20, 22, 25, 30, 32, 35,  40, 45, 50]])
      neural_sum = np.concatenate((trainTest[dataDirs[x]+'_train'], trainTest[dataDirs[x]+'_test']), axis=1)
      neural_test = np.array(list(set(map(tuple, neural_sum.T)) - set(map(tuple, maxtrainset.T)))).T

#      tr = np.setdiff1d(trainTest[dataDirs[x]+'_train'], nueral_train)  # Original set - trainingSamples
      trainTest[dataDirs[x]+'_test']= neural_test  # Samples for testing
      print("After", trainTest[dataDirs[x]+'_test'])

    scipy.io.savemat('local/'+dataDirs[x]+'_trainTest.mat', trainTest)
    scipy.io.savemat('local/'+dataDirs[x]+'_singleLetters.mat', singleLetterDat) 
    
 #  args['sentencesFile_0'] = rootDir+'Datasets/'+dataDirs[x]+'/sentences.mat'
 #  args['singleLettersFile_0'] = rootDir+'Datasets/'+dataDirs[x]+'/singleLetters.mat'
 #  args['sentencesFile_0'] ='C:/IamOnline/neuralCubeStruct.mat'

    args['sentencesFile_0'] = 'local/'+dataDirs[x]+'_sentences.mat' 
    args['singleLettersFile_'+str(x)] = 'local/'+dataDirs[x]+'_singleLetters.mat'
    
    args['labelsFile_0'] = rootDir+'RNNTrainingSteps/Step2_HMMLabels/'+cvPart+'/'+dataDirs[x]+'_timeSeriesLabels.mat'
    args['syntheticDatasetDir_0'] = rootDir+'RNNTrainingSteps/Step3_SyntheticSentences/'+cvPart+'/'+dataDirs[x]+'_syntheticSentences/'
  #  args['cvPartitionFile_0'] = rootDir+'RNNTrainingSteps/trainTestPartitions_'+cvPart+'.mat'
    args['cvPartitionFile_'+str(x)] = 'local/'+dataDirs[x]+'_trainTest.mat'
    args['sessionName_0'] = dataDirs[x]

    args['inferenceOutputFileName'] = inferenceSaveDir + '/' + dataDirs[x] + '_inferenceOutputs.mat'
    args['inferenceInputLayer'] = x
#    args['inferenceInputLayer'] = 5
    
    #For the top GRU layer, how many bins to skip for each update (the top layer runs at a slower frequency)                             
    args['skipLen'] = 1
    #number of units in each GRU layer
   # args['nUnits'] = 512
    #Can be 'unidrectional' (causal) or 'bidirectional' (acausal)                              
 #   args['directionality'] = 'unidirectional'
    #l2 regularization cost                             
    args['l2scale'] = 1e-6   

    
    #instantiate the RNN model
    rnnModel = charSeqRNN(args=args)

    #evaluate the RNN on the held-out data
    outputs = rnnModel.inference()
    
    #reset the graph to make space for the next dataset
    tf.reset_default_graph()


 
Processing dataset IamOnline1
Modifying to Iamonline data
Working on  IamOnline1
Before Iamonline:  [[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
   18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
   36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
   54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
   72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
   90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
  108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
  126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
  144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
  162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
  180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
  198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
  216 217 

Sentence File (IamOnline) is  local/IamOnline2_sentences.mat
IamOnline2
local/IamOnline1_trainTest.mat
{'__header__': b'MATLAB 5.0 MAT-file Platform: nt, Created on: Sat Jan 29 16:55:43 2022', '__version__': '1.0', '__globals__': [], 't5.2019.05.08_train': array([[  0,   1,   3,   4,   6,   7,   8,   9,  10,  11,  12,  13,  14,
         15,  16,  18,  19,  20,  22,  23,  24,  25,  26,  27,  28,  29,
         30,  32,  35,  36,  37,  38,  40,  41,  42,  43,  44,  45,  46,
         47,  48,  49,  50,  51,  52,  53,  55,  56,  57,  58,  59,  60,
         62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,
         75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
         88,  89,  92,  93,  94,  95,  96,  97,  99, 100, 101]]), 't5.2019.05.08_test': array([[ 2,  5, 21, 33, 34, 39, 54, 90, 91, 98]]), 't5.2019.11.25_train': array([[ 0,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17,
        18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 30, 32, 35, 36, 37, 38,

KeyError: 'IamOnline2_train'

In [ ]:
#This cell loads the outputs produced above and computes character error counts and word error counts.
from characterDefinitions import getHandwritingCharacterDefinitions
from characterDefinitionsOrig import getHandwritingCharacterDefinitionsOrig
from rnnEval import evaluateRNNOutput, rnnOutputToKaldiMatrices
import warnings

#this stops scipy.io.savemat from throwing a warning about empty entries
warnings.simplefilter(action='ignore', category=FutureWarning)

allErrCounts = []

for x in range(len(dataDirs)):
    print('-- ' + dataDirs[x] + ' --')
    #defines the list of all 31 characters and what to call them
    if "IamOnline" in dataDirs[x]:
       charDef = getHandwritingCharacterDefinitions()
    else:
       charDef = getHandwritingCharacterDefinitionsOrig() 
    
    #Load up the outputs, which are frame-by-frame probabilities. 
    outputs = scipy.io.loadmat(inferenceSaveDir + '/' + dataDirs[x] + '_inferenceOutputs.mat')
    
    if "IamOnline" in dataDirs[x]:
  #    sentenceDat = scipy.io.loadmat(rootDir+'Datasets/'+'t5.2019.05.08'+'/sentences.mat')# Template is required
      sentence1 = scipy.io.loadmat(rootDir+'Datasets/'+dataDirs[x]+'/'+'neuralCubeStruct_IamOnline.mat')
      sentenceDat = scipy.io.loadmat(rootDir+'Datasets/'+dataDirs[x]+'/'+'neuralCubeStruct_IamOnline.mat')
  #    sentenceDat['neuralActivityCube'] = sentence1['neuralActivityCube']
  #    sentenceDat['intendedText'] = sentence1['intendedText']
  #    sentenceDat['sentencePrompt'] = sentence1['sentencePrompt']
  #    sentenceDat['numTimeBinsPerSentence'] = sentence1['numTimeBinsPerSentence']
      sentenceDat['blockList'] = []
#      sentenceDat['trainPartitionIdx'] = sentence1['trainPartitionIdx']
#      sentenceDat['testPartitionIdx'] = sentence1['testPartitionIdx'] 
    else:
      sentenceDat = scipy.io.loadmat(rootDir+'Datasets/'+dataDirs[x]+'/sentences.mat')
   
    
    #Convert the outputs into character sequences (with simple thresholding) & get word/character error counts.
    errCounts, decSentences = evaluateRNNOutput(outputs['outputs'], 
                                        sentenceDat['numTimeBinsPerSentence']/2 + 50, 
                                        sentenceDat['sentencePrompt'], 
                                        charDef, 
                                        charStartThresh=0.20, 
                                        charStartDelay=15)
    # Best results for IamOnline is charStartThresh=0.20 or charStartDelay=25
    
#    errCounts, decSentences = evaluateRNNOutput(outputs['outputs'], 
#                                        sentenceDat['numTimeBinsPerSentence']/2 + 50, 
#                                        sentenceDat['sentencePrompt'], 
#                                        charDef, 
#                                        charStartThresh=0.20, 
#                                        charStartDelay=20)    
    #save decoded sentences, character error rates and word error rates for later summarization
    saveDict = {}
    saveDict['decSentences'] = decSentences
    saveDict['trueSentences'] = sentenceDat['sentencePrompt']
    saveDict.update(errCounts)
    
    scipy.io.savemat(inferenceSaveDir + '/' + dataDirs[x] + '_errCounts.mat', saveDict)
    
    #print results for the validation sentences
   # cvPartFile = scipy.io.loadmat(rootDir+'RNNTrainingSteps/trainTestPartitions_'+cvPart+'.mat')
    cvPartFile = scipy.io.loadmat('local/'+dataDirs[x]+'_trainTest.mat')
    valIdx = cvPartFile[dataDirs[x]+'_test']
    
 #   if dataDirs[x] == "IamOnline":   # Nithin change
 #      valIdx = cvPartFile['t5.2019.05.08'+'_test']
 #   else:
 #      valIdx = cvPartFile[dataDirs[x]+'_test']
    
    if len(valIdx)==0:
        print('No validation sentences for this session.')
        print('  ')
        continue
            
    valAcc = 100*(1 - np.sum(errCounts['charErrors'][valIdx]) / np.sum(errCounts['charCounts'][valIdx]))

    print('Character error rate for this session: %1.2f%%' % float(100-valAcc))
    print('Below is the decoder output for all validation sentences in this session:')
    print(' ')
    
    for v in np.squeeze(valIdx):
        trueText = sentenceDat['sentencePrompt'][v,0][0]
        trueText = trueText.replace('>',' ')
        trueText = trueText.replace('~','.')
        trueText = trueText.replace('#','')
        
        print('#' + str(v) + ':')
        print('True:    ' + trueText)
        print('Decoded: ' + decSentences[v])
        print(' ')
   
    #put together all the error counts from all sessions so we can compute overall error rates below
    allErrCounts.append(np.stack([errCounts['charCounts'][valIdx],
                             errCounts['charErrors'][valIdx],
                             errCounts['wordCounts'][valIdx],
                             errCounts['wordErrors'][valIdx]],axis=0).T)
        

In [ ]:
#Summarize character error rate and word error rate across all sessions.
concatErrCounts = np.squeeze(np.concatenate(allErrCounts, axis=0))
cer = 100*(np.sum(concatErrCounts[:,1]) / np.sum(concatErrCounts[:,0]))
wer = 100*(np.sum(concatErrCounts[:,3]) / np.sum(concatErrCounts[:,2]))

print('Character error rate: %1.2f%%' % float(cer))
print('Word error rate: %1.2f%%' % float(wer))